# Init

In [ ]:
import os

import matplotlib.pylab as plt

import numpy as np
import pandas as pd
import seaborn as sns

from luescher_nd.database.utilities import read_table
from luescher_nd.zeta.zeta2d import Zeta2D

In [ ]:
%load_ext blackcellmagic

# Comparison with Tom

Init the zeta functions

In [ ]:
N = 100
z_spherical = Zeta2D(N, spherical=True)
z_cartesian = Zeta2D(N, spherical=False)

Plot to compare with Tom

In [ ]:
fig, ax = plt.subplots(figsize=(4,2), dpi=250)

tol = 40.0

x = np.linspace(-4, 10, 987)

linestyles = {
    "spherical": "-", 
    "cartesian": "--"
}

for z, key in zip([z_spherical, z_cartesian], ["spherical", "cartesian"]):
    y = z(x)
    y[y>tol] = np.inf
    y[y<-tol] = -np.inf

    ax.plot(x, y, ls=linestyles[key], label=key, lw=1)

ax.axhline(0, color="gray", lw=.5, ls="-", zorder=-1, alpha=.5)
for pole in [0, 1, 2, 4, 5, 8, 9]:
    ax.axvline(pole, color="gray", lw=.5, ls="-", zorder=-1, alpha=.5)

sns.despine()

ax.legend(loc="upper left", bbox_to_anchor=(1,1), frameon=False)

ax.set_ylim(-20, 20)
ax.set_xlim(-4, 10)

ax.set_xlabel("$x$")
ax.set_ylabel("$S_2(x)$")

ax.set_title(f"$N={N}$", fontsize=10)

plt.show()

# Plot database results

In [ ]:
ROOT = os.path.abspath("../data/two-d")
db = os.path.join(ROOT, "db-contact-fv-c-parity.sqlite")

In [ ]:
mass = 4.758
mu = mass / 2

In [ ]:
def ere(y, x):
    return y / np.pi**2 + 1 / np.pi * np.log(x)

In [ ]:
df = read_table(database=db, drop_comment=False).drop(columns="y")

df = (
    pd.merge(
        df,
        df.comment.str.extract(
            "spherical=(?P<spherical>[01]{1})&a0=(?P<a0>[0-9\.]+)&Lambda=(?P<Lambda>[0-9\.]+)"
        ),
        how="inner",
        left_index=True,
        right_index=True,
    )
    .astype(dtype={"spherical": int, "a0": float, "Lambda": float})
    .astype({"spherical": bool})
)
zeta = {True: z_spherical, False: z_cartesian}


df["y"] = df.apply(
    lambda row: ere(zeta[row["spherical"]](row["x"]), row["x"])[0], axis=1
)
df = df.dropna()

a0 = df.a0.unique()

df = df.drop(columns=["comment", "date"])
df["epsilon"] = df.epsilon.round(3)

tf = df.query("y < 5 and y > -5 and epsilon < 0.1 and L == 1").sort_values(
    ["spherical", "n1d", "nstep", "nlevel"]
)

tf.head()

In [ ]:
grid = sns.FacetGrid(
    data=tf,
    row="spherical",
    col="nstep",
    hue="epsilon",
    legend_out=True,
    margin_titles=True,
    col_order=["1", "4", "$\\infty$"],
    hue_kws=dict(marker=["o", "s", "v", "^", "d"])
)

grid.map(plt.plot, "x", "y", ms=3, ls=":", marker="s", lw=0.5).add_legend(title=r"$\epsilon$ [fm]")

x = np.linspace(-0.1, 20, 200)

for ax in grid.axes.flatten():
    ax.set_ylim(-1, 3)

    ax.axhline(0, color="black", lw=0.5, ls="-", zorder=-1)
    ax.axvline(0, color="black", lw=0.5, ls="-", zorder=-1)
    ax.plot(
        x,
        2 / np.pi * np.log(a0 * 2 * np.pi * np.sqrt(x) / tf.L.unique()),
        color="black",
        ls="--",
        lw="1",
        label="Analytic",
        zorder=-1,
    )

grid.set_ylabels("$\cot(\delta_0(x))$")
grid.set_xlabels("$x$")

grid.fig.suptitle(f"$L = {tf.L.unique()[0]}$ [fm], $a_0 = {tf.a0.unique()[0]}$ [fm]", y=1.06)

grid.fig.set_dpi(250)
grid.fig.set_figheight(4)

In [ ]:
cf = df.groupby(["epsilon", "L", "nstep", "spherical"])[["contact_strength"]].mean().reset_index()#.query("L == 1")

cf["y"] = -1./cf["contact_strength"]

cf.head()

In [ ]:
grid = sns.FacetGrid(
    data=cf,
    row="spherical",
    col="nstep",
    hue="L",
    legend_out=True,
    margin_titles=True,
    col_order=["1", "4", "$\\infty$"],
    hue_kws=dict(marker=["o", "s", "v", "^", "d"]),
)

grid.map(plt.plot, "epsilon", "y", ms=3, ls=":", marker="s", lw=0.5)

x = np.linspace(-0.1, 20, 200)

grid.set_ylabels(r"$-1/c_0(\epsilon)$ [fm$^{-1}$]")
grid.set_xlabels("$\epsilon$ [fm]")

grid.fig.suptitle(f"$a_0 = {df.a0.unique()[0]}$ [fm]", y=1.06)

eps = np.linspace(1.0e-2, 9.0e-1, 100)
Lambda = np.pi / eps
y1 = 2 * mu / np.pi / 2 * np.log(a0 * Lambda)
Gmphl2 = -0.1728274509745820501957409
y2 = y1 - 2 * mu / np.pi**2 * Gmphl2

for ax in grid.axes.flatten():
    ax.set_xscale("log")
    ax.set_xlim(1.0e-2, 9.0e-1)
    ax.plot(
        eps,
        y1,
        label=r"$\frac{\mu}{\pi} \log\left(\frac{\pi}{\epsilon}a_0\right)$",
        ls="-",
        color="blue",
        lw=.5,
        zorder=-1
    )
    ax.plot(
        eps,
        y2,
        label=r"$\frac{\mu}{\pi} \log\left(\frac{\pi}{\epsilon}a_0\right) "
        r"- \frac{2\mu}{\pi^2}\left(G - \frac{\pi}{2} \log(2)\right)$",
        ls="-",
        color="green",
        lw=.5,
        zorder=-1
    )

grid.fig.set_dpi(250)
grid.fig.set_figheight(4)

plt.subplots_adjust()

leg = ax.legend(fontsize=10, frameon=False, loc="upper left", bbox_to_anchor=(1.1, 1.5), title="$L$ [fm]")


In [ ]:
def c_analytic(epsilon, spherical):
    Lambda = np.pi / epsilon
    y1 = 2 * mu / np.pi / 2 * np.log(a0 * Lambda)
    Gmphl2 = -0.1728274509745820501957409
    if not spherical:
        y1 -= 2 * mu / np.pi**2 * Gmphl2
    return y1

def error(actual, expected):
    return abs(actual + expected)/expected * 100

In [ ]:
query = "nstep == '$\\infty$'"
ttf = cf.query(query).copy()
ttf["y_analytic"] = ttf.apply(lambda row: c_analytic(row["epsilon"], row["spherical"]),
    axis=1,
)
ttf["diff"] = (ttf["y_analytic"] -  ttf["y"]).abs()/ttf["y_analytic"] 
ttf.head()

In [ ]:
grid = sns.FacetGrid(
    data=ttf,
    col="spherical",
    hue="L",
    legend_out=True,
    margin_titles=True,
    hue_kws=dict(marker=["o", "s", "v", "^", "d"]),
)

grid.map(plt.plot, "epsilon", "diff", ls="--", lw=0.5, marker="o", ms=1)

grid.fig.set_dpi(250)
grid.fig.set_figheight(3)

for ax in grid.axes.flatten():
    ax.set_yscale("log")
    ax.set_xscale("log")
    ax.set_xlim(1.0e-2, 9.0e-1)
    ax.set_ylim(1.0e-4, 1.1e-1)

plt.subplots_adjust()
grid.fig.suptitle(query, fontsize=12, y=1.06)

ax.legend(
    fontsize=10,
    frameon=False,
    loc="upper left",
    bbox_to_anchor=(1.1, 0.8),
    title="$L$ [fm]",
)
grid.set_xlabels("$\epsilon$ [fm]")
grid.set_ylabels(r"$ \left|\Delta\frac{-1}{c_0(\epsilon)}\right|$ [%]")

plt.show()